# Кредитное моделирование риска дефолта

Классическая ML-задача бинарной классификации для оценки вероятности дефолта клиента на горизонте 12 месяцев. Ноутбук оформлен как реплицируемый конвейер: от загрузки данных до финального скоринга и интерпретации.

## 1. Обзор проекта
- **Цель:** предсказать вероятность дефолта клиента (PD) для поддержки кредитных решений.
- **Таргет:** `flag` (0 — без дефолта, 1 — дефолт).
- **Метрики:** ROC-AUC (основная), PR-AUC, F1/Recall (по бизнес-порогам).
- **Результаты:** обученные артефакты модели, отчет о валидации, пайплайн инференса/скоринга.

In [ ]:
%pip install -q --upgrade pip setuptools wheel

%pip install -q \
  "numpy==1.26.4" \
  "pandas==2.2.2" \
  "scikit-learn==1.3.2" \
  "matplotlib==3.8.4" \
  "seaborn==0.13.2" \
  "ydata-profiling" \
  "xgboost==2.0.3" \
  "lightgbm==4.3.0" \
  "catboost==1.2.5" \
  "lightautoml" \
  "optuna==3.6.1" \
  "joblib>=1.3" \
  "tqdm>=4.66" \
  "ipywidgets==8.1.2" \
  "jupyterlab-widgets==3.0.10" \
  "nltk==3.9.1" \
  "gensim==4.3.3" \
  "transformers==4.44.2" \
  "sentencepiece>=0.1.99" \
  "tokenizers<0.20"\
  "pyarrow"


In [ ]:
import sys, platform
import pandas as pd, numpy as np, sklearn, matplotlib, seaborn as sns, ydata_profiling as ydata
import xgboost, lightgbm, catboost
import lightautoml as lama

print("Python:", sys.version.split()[0], "|", platform.platform())
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)
print("matplotlib:", matplotlib.__version__)
print("seaborn:", sns.__version__)
print("xgboost:", xgboost.__version__)
print("lightgbm:", lightgbm.__version__)
print("catboost:", catboost.__version__)
print("lightautoml:", lama.__version__)
print("ydata_profiling:", ydata.__version__)

## Содержание
1. Обзор проекта
2. Данные и словарь признаков
3. Профилирование данных (ydata-profiling)
4. Обработка пропусков и выбросов
5. Генерация и отбор признаков (5 вариантов датасета)
6. Разбиение данных: стратегии (для каждого варианта)
7. Бейзлайн по датасетам
8. AutoML
9. Интерпретация и сдвиги данных
10. Инференс и подготовка сабмита


## 2. Данные и словарь признаков
- **Файлы:**
  - `train_data_*.pq` — табличные фичи по клиентам.
  - `train_target.csv` — целевая переменная `flag`.
  - `models/light_automl.pkl` — сохраненная модель.
- **Идентификатор:** `id` (или иной уникальный ключ, при наличии).
- **Словарь признаков:**
  - Описать типы: числовые, категориальные, даты, агрегаты.
  - Бизнес-смысл ключевых полей (доход, обороты, просрочки, лимиты и т.д.).
- **Качество данных:**
  - Доли пропусков, выбросы, сдвиги распределений между train/valid/test.


In [1]:
import warnings
import psutil, os, glob
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
import lightgbm as lgb
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_zipped_file


from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (
    AdaBoostClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    IsolationForest,
    RandomForestClassifier,
)
from sklearn.exceptions import DataConversionWarning
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    classification_report,
    confusion_matrix,
    log_loss,
    roc_auc_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    StratifiedKFold,
    cross_validate,
    train_test_split,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


warnings.filterwarnings("ignore") 

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
PATH = "train_data"
OUT_FILE = "train_data_full.parquet"

files = sorted(
    glob.glob(os.path.join(PATH, "train_data_*.pq")),
    key=lambda p: int(os.path.splitext(os.path.basename(p))[0].split("_")[-1])
)

mins, maxs, is_intlike = {}, {}, {}

for f in tqdm(files, desc="Pass1: scan stats"):
    df = pd.read_parquet(f)
    for c in df.columns:
        s = df[c]

        if s.dropna().empty:
            continue
        vmin, vmax = s.min(skipna=True), s.max(skipna=True)
        mins[c] = vmin if c not in mins else min(mins[c], vmin)
        maxs[c] = vmax if c not in maxs else max(maxs[c], vmax)

        if c not in is_intlike:
            is_intlike[c] = True
        if pd.api.types.is_float_dtype(s):
            frac = (s.dropna() - s.dropna().round()).abs().max()
            if pd.notna(frac) and frac > 1e-9:
                is_intlike[c] = False

def pick_nullable_dtype(vmin, vmax, intlike=True):
    if not intlike:
        return None  
    if vmin >= 0:
        if vmax <= 1:
            return "boolean"      
        elif vmax <= 255:
            return "UInt8"
        elif vmax <= 65535:
            return "UInt16"
        elif vmax <= 4294967295:
            return "UInt32"
        else:
            return "UInt64"
    else:
        if -128 <= vmin and vmax <= 127:
            return "Int8"
        elif -32768 <= vmin and vmax <= 32767:
            return "Int16"
        elif -2147483648 <= vmin and vmax <= 2147483647:
            return "Int32"
        else:
            return "Int64"

target_dtypes = {}
for c in maxs.keys(): 
    dt = pick_nullable_dtype(mins.get(c, 0), maxs[c], is_intlike.get(c, False))
    if dt is not None:
        target_dtypes[c] = dt


writer = None
cols = None

def cast_series(s: pd.Series, dtype: str) -> pd.Series:
    if pd.api.types.is_float_dtype(s):
        s = s.where(s.isna(), s.round())
    return s.astype(dtype)

try:
    for i, f in enumerate(tqdm(files, desc="Pass2: write")):
        df = pd.read_parquet(f)

        if cols is None:
            cols = list(df.columns)

        for c, dt in target_dtypes.items():
            if c in df.columns:
                df[c] = cast_series(df[c], dt)

        df = df.reindex(columns=cols)

        table = pa.Table.from_pandas(df, preserve_index=False)
        if writer is None:
            writer = pq.ParquetWriter(OUT_FILE, table.schema, compression="zstd")
        writer.write_table(table)

        del df, table
finally:
    if writer is not None:
        writer.close()

In [ ]:
df = pd.read_parquet("train_data_full.parquet", engine="pyarrow")

In [ ]:
vm = psutil.virtual_memory()
print("Total GB:", round(vm.total/1e9,2), "Used GB:", round((vm.total-vm.available)/1e9,2), "Avail GB:", round(vm.available/1e9,2))

proc = psutil.Process(os.getpid())
print("Proc RSS GB:", round(proc.memory_info().rss/1e9,2))

## 3. Профилирование данных (ydata-profiling)
- Генерация полного отчёта профилирования для `train_data_*.pq` + `train_target.csv`.
- Блоки отчёта: overview, missingness, distributions, correlations, interactions, warnings.
- Фиксация ключевых находок: дисбаланс классов, проблемные фичи, потенциальные утечки.
- Ссылки/экспорт: HTML-отчёт, краткое резюме выводов.


In [ ]:
# profile = ProfileReport(
#     df, title="Profile Report of the CreditRisk", explorative=True
# )

In [ ]:
# profile

## 4. Обработка пропусков и выбросов
- Стратегии для числовых признаков: медиана/спец-коды/модели восстановления.
- Для категориальных: частотная замена/`Unknown`.
- Обработка выбросов: винсоризация/клиппинг/лог-трансформы.
- Фиксация принятых правил, чтобы использовать их на инференсе.


## 5. Генерация и отбор признаков: варианты датасетов
- Базовые преобразования: агрегаты, лог-/норм-трансформы, масштабирование.
- Кодирование категорий: Target/WOE/Count/Frequency, One-Hot (по необходимости).
- Отбор: важности, устойчивость, мультиколлинеарность, бизнес-логика.
- Ниже — 5 подвариантов подготовки датасета (будут использоваться далее для разных стратегий разбиения и бейзлайна).


### 5.1 Вариант датасета A
- Описание преобразований и отборов для варианта A.
- Список ключевых фичей и логика их формирования.


### 5.2 Вариант датасета B
- Отличия от A, альтернативные кодирования/агрегаты.
- Риск утечек и как он предотвращается.


### 5.3 Вариант датасета C
- Специализированные признаки (временные окна/отношения/нормализации).
- Обоснование выбора.


### 5.4 Вариант датасета D
- Комбинации фичей/интеракции, редукция измерений (при необходимости).
- Критерии отбора.


### 5.5 Вариант датасета E
- Смесь подходов A–D, компромисс по качеству/скорости.
- Список исключённых признаков и причины.


## 6. Разбиение данных: стратегии
- Общая логика: повторяем для каждого варианта датасета (A–E).
- Варианты:
  - Простое: `train`/`val`/`test` (стратифицированно по `flag`).
  - С семплированием: undersampling/oversampling/SMOTE на `train`.
- Фиксировать сиды, сохранять индексы/идентификаторы разбиений.


### 6.A Разбиение для датасета A
- Вариант 1: `train/val/test` (стратификация).
- Вариант 2: семплирование на `train` (указать метод и параметры).


### 6.B Разбиение для датасета B
- Вариант 1: `train/val/test` (стратификация).
- Вариант 2: семплирование на `train` (указать метод и параметры).


### 6.C Разбиение для датасета C
- Вариант 1: `train/val/test` (стратификация).
- Вариант 2: семплирование на `train` (указать метод и параметры).


### 6.D Разбиение для датасета D
- Вариант 1: `train/val/test` (стратификация).
- Вариант 2: семплирование на `train` (указать метод и параметры).


### 6.E Разбиение для датасета E
- Вариант 1: `train/val/test` (стратификация).
- Вариант 2: семплирование на `train` (указать метод и параметры).


## 7. Бейзлайн по датасетам
- Для каждого варианта (A–E) обучить простой бейзлайн.
- Фиксировать метрики на `val`/`test`, сохранять конфигурации и сиды.


### 7.A Бейзлайн для датасета A
- Модель(и): Logistic Regression / LightGBM (минимальная настройка).
- Метрики: ROC-AUC, PR-AUC, F1/Recall (при пороге).


### 7.B Бейзлайн для датасета B
- Модель(и): Logistic Regression / LightGBM.
- Метрики и сравнение с A.


### 7.C Бейзлайн для датасета C
- Модель(и): Logistic Regression / LightGBM.
- Метрики и сравнение с A/B.


### 7.D Бейзлайн для датасета D
- Модель(и): Logistic Regression / LightGBM.
- Метрики и сравнение с предыдущими.


### 7.E Бейзлайн для датасета E
- Модель(и): Logistic Regression / LightGBM.
- Итоговая таблица сравнения по всем вариантам.


## 8. AutoML
- Запуск AutoML (например, LightAutoML/FLAML/AutoGluon) на выбранных вариантах датасетов.
- Ограничения по времени/ресурсам, критерии ранней остановки.
- Сводка лучших конфигураций и сравнение с бейзлайнами.


## 9. Интерпретация и сдвиги данных
- Важности признаков: Gain/Split, Permutation Importance.
- Локальные объяснения: SHAP/ICE для кейсов.
- Сдвиги: PSI/дрифт фичей между train/valid/test.
- Проверка справедливости (bias) при наличии чувствительных атрибутов.


## 10. Инференс и подготовка сабмита
- Единый конвейер препроцессинга и модели для прод/оффлайн скоринга.
- Формат предсказаний: `client_id`, `score` (PD) и, при необходимости, `class` по бизнес-порогу.
- Экспорт сабмита/отчёта, контроль версий и дат.
